In [92]:
import os
import json
import pandas as pd
import traceback

In [93]:
import sys
print(sys.executable)

C:\Users\anubh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe


In [94]:
!pip install langchain openai -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\anubh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [95]:
%pip install langchain_community -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\anubh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [96]:
import openai
from langchain.chat_models import ChatOpenAI

In [97]:
from dotenv import load_dotenv
load_dotenv() # Load environment variables from .env file

True

In [98]:
KEY = os.getenv("OPENAI_API_KEY")
if not KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")
else :
    print("API Key is set.")

API Key is set.


In [ ]:

llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [100]:
%pip install python-dotenv streamlit PyPDF2 -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\anubh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [101]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

Designing the Prompt template

In [102]:
RESPONSE_JSON = {
    "questions": [
        {
            "question": "What is the capital of France?",
            "options": ["A) Paris", "B) London", "C) Berlin", "D) Madrid"],
            "answer": "A"
        },
        {
            "question": "What is the largest planet in our solar system?",
            "options": ["A) Earth", "B) Jupiter", "C) Saturn", "D) Mars"],
            "answer": "B"
        }
    ]
}

In [103]:
TEMPLATE = """
You are an expert educator with deep knowledge in {topic} at a {level} level.
You also understand human psychology and engagement design, enabling you to create fun, thought-provoking quizzes.

Your task is to generate exactly {number_of_questions} quiz questions based on the following source material:
"{text}". Make sure to cover key concepts from the text and the questions should vary in structure (e.g., multiple choice, true/false, fill-in-the-blank), if appropriate.
The questions should not be repeatable and should be designed to test understanding of the material.

### Requirements:
1. Match the tone: **{tone}**
2. Match the difficulty: **{level}** level
3. Ensure coverage of key concepts from the text.
4. Include a clear explanation for each correct answer.
5. Questions should vary in structure (e.g., multiple choice, true/false, fill-in-the-blank), if appropriate.

### Format:
Return the quiz in this exact JSON format:
{response_json}

Make sure the JSON is syntactically valid and structured precisely.
Do not add any extra commentary outside the JSON.
"""


In [104]:
quiz_generator_prompt = PromptTemplate(
    input_variables=["text","number_of_questions","topic","level","tone","response_json"],
    template = TEMPLATE
)

Chaining

In [105]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_generator_prompt,
    output_key="quiz",
    verbose=True
)
"""
output_key lets you control the key name for the chain's output in the result dictionary, making it easier to access the generated content.
"""

"\noutput_key lets you control the key name for the chain's output in the result dictionary, making it easier to access the generated content.\n"

Evaluating the generated quiz

In [106]:
#to checck if the generated quiz is valid or not
TEMPLATE2 = """
You are an expert English grammarian, cognitive learning specialist, and educational content editor.

Your task is to review the following multiple-choice quiz designed for {topic} students.

### Instructions:
1. **Evaluate the overall language, tone, and complexity** of the quiz.
2. **Assess if the questions align** with the cognitive and analytical level appropriate for {topic} students.
3. **Keep the complexity analysis within 50 words.**
4. If any questions are too hard, too easy, poorly worded, or misaligned:
   - **Revise only those specific questions.**
   - Adjust the tone and phrasing to better suit the target student's ability.
   - Preserve the original structure (e.g., multiple-choice format).

### Input Quiz:
{quiz}

### Output Format:
- **Complexity Analysis (<= 50 words)**:
  ...
- **Updated Questions (if any)**:
  ...
"""



In [107]:
quiz_eval_prompt = PromptTemplate(
    input_variables=["quiz", "topic"],
    template=TEMPLATE2
    )

In [108]:
review_chain = LLMChain(
    llm=llm,
    prompt=quiz_eval_prompt,
    output_key="review",
    verbose=True
)

Connecting the chains using sequential Chains

In [109]:
generate_eval_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number_of_questions", "topic", "level", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

Passing it data via a file

In [110]:
file_path = r"D:\VS_Code_WorkSpaces\GenAI\data.txt"

In [111]:
file_path

'D:\\VS_Code_WorkSpaces\\GenAI\\data.txt'

In [112]:
with open(file_path, 'r') as file:
    data = file.read()

In [113]:
print(data)

AI automation refers to the use of Artificial Intelligence (AI) technologies to automate complex tasks and processes that typically require human intelligence. It goes beyond traditional rule-based automation (like Robotic Process Automation - RPA) by enabling systems to learn, adapt, and make informed decisions based on data.

Key Aspects of AI Automation:
Intelligent Decision-Making: AI automation leverages machine learning (ML) and deep learning to analyze data, identify patterns, and make more accurate predictions and decisions than traditional automation.
Handling Unstructured Data: Unlike traditional automation, AI automation can handle unstructured data such as text, images, and audio, using technologies like Natural Language Processing (NLP) and computer vision.
Continuous Learning: AI systems continuously learn and improve over time as they process new data, becoming more efficient and accurate.
Scalability: AI automation can scale quickly to handle increasing volumes of work 

In [114]:
#serializing the py dict into a json file
json.dumps(RESPONSE_JSON)

'{"questions": [{"question": "What is the capital of France?", "options": ["A) Paris", "B) London", "C) Berlin", "D) Madrid"], "answer": "A"}, {"question": "What is the largest planet in our solar system?", "options": ["A) Earth", "B) Jupiter", "C) Saturn", "D) Mars"], "answer": "B"}]}'

LangChain callbacks are hooks or listeners that let you track and log what happens during the execution of LLM chains, tools, and agents.

Think of them as a "behind-the-scenes monitor" that:

Captures intermediate steps

Measures token usage, cost, and latency

Logs prompts and outputs

Helps in debugging and optimization

In [115]:
#to track token usage in langchain
with get_openai_callback() as cb:
    try:
        response = generate_eval_chain.invoke({
            "text": data,
            "number_of_questions": 5,
            "topic": "AI Automation",
            "level": "intermediate",
            "tone": "engaging and educational",
            "response_json": json.dumps(RESPONSE_JSON)
        })
        print("Response:", response)
    except Exception as e:
        print("An error occurred:", e)
        traceback.print_exc()



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert educator with deep knowledge in AI Automation at a intermediate level.
You also understand human psychology and engagement design, enabling you to create fun, thought-provoking quizzes.

Your task is to generate exactly 5 quiz questions based on the following source material:
"AI automation refers to the use of Artificial Intelligence (AI) technologies to automate complex tasks and processes that typically require human intelligence. It goes beyond traditional rule-based automation (like Robotic Process Automation - RPA) by enabling systems to learn, adapt, and make informed decisions based on data.

Key Aspects of AI Automation:
Intelligent Decision-Making: AI automation leverages machine learning (ML) and deep learning to analyze data, identify patterns, and make more accurate predictions and decisions than traditional automation.
Handling Unstructured Data: Unlike

An error occurred: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-or-v1*************************************************************ff7e. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


Traceback (most recent call last):
  File "C:\Users\anubh\AppData\Local\Temp\ipykernel_11332\2110230405.py", line 4, in <module>
    response = generate_eval_chain.invoke({
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\anubh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain\chains\base.py", line 167, in invoke
    raise e
  File "C:\Users\anubh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain\chains\base.py", line 157, in invoke
    self._call(inputs, run_manager=run_manager)
  File "C:\Users\anubh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain\chains\sequential.py", line 107, in _call
    outputs = chain(known_values, return_only_outputs=True, callbacks=callbacks)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [116]:
print(cb)

Tokens Used: 0
	Prompt Tokens: 0
		Prompt Tokens Cached: 0
	Completion Tokens: 0
		Reasoning Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [ ]:
response_json = response.get('quiz')

In [ ]:
response_json = json.loads(response_json)

In [ ]:
quiz_table_data = []
for key, value in response_json.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

quiz_table_data

In [ ]:
quiz = pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("AIAutomation_quiz.csv", index = False)

In [ ]:
from datetime import datetime
datetime_str = datetime.now().strftime('%m_%d_%Y_%H_%M_%S')
datetime_str #saving the file in the logger with the current date and time

'07_02_2025_15_47_49'